In [ ]:
# !pip install tensorflow==1.15.0

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import InputLayer, Input
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.callbacks import TensorBoard
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping

import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors
import matplotlib as mpl

In [ ]:
randomState = 42
np.random.seed(randomState)
tf.set_random_seed(randomState)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df5 = pd.read_csv('/content/gdrive/MyDrive/SoilMoisture/Data/c_5_data_soil_5.csv')
df10 = pd.read_csv('/content/gdrive/MyDrive/SoilMoisture/Data/c_5_data_soil_10.csv')
df20 = pd.read_csv('/content/gdrive/MyDrive/SoilMoisture/Data/c_5_data_soil_20.csv')
df50 = pd.read_csv('/content/gdrive/MyDrive/SoilMoisture/Data/c_5_data_soil_50.csv')
df100 = pd.read_csv('/content/gdrive/MyDrive/SoilMoisture/Data/c_5_data_soil_100.csv')

df5 = df5.rename(columns={'In-Situ':'InSitu'})
df10 = df10.rename(columns={'In-Situ':'InSitu'})
df20 = df20.rename(columns={'In-Situ':'InSitu'})
df50 = df50.rename(columns={'In-Situ':'InSitu'})
df100 = df100.rename(columns={'In-Situ':'InSitu'})

def replace_missing(attribute):
    return attribute.interpolate(inplace=True)

replace_missing(df5['NDVI'])
replace_missing(df5['EVI'])
replace_missing(df5['LSTDay'])
replace_missing(df5['LSTNight'])

replace_missing(df10['NDVI'])
replace_missing(df10['EVI'])
replace_missing(df10['LSTDay'])
replace_missing(df10['LSTNight'])

replace_missing(df20['NDVI'])
replace_missing(df20['EVI'])
replace_missing(df20['LSTDay'])
replace_missing(df20['LSTNight'])

replace_missing(df50['NDVI'])
replace_missing(df50['EVI'])
replace_missing(df50['LSTDay'])
replace_missing(df50['LSTNight'])

replace_missing(df100['NDVI'])
replace_missing(df100['EVI'])
replace_missing(df100['LSTDay'])
replace_missing(df100['LSTNight'])


df5 = df5.dropna()
df10 = df10.dropna()
df20 = df20.dropna()
df50 = df50.dropna()
df100 = df100.dropna()

In [ ]:
df5.pop("BD")
df10.pop("BD")
df20.pop("BD")
df50.pop("Precip")
df100.pop("BD")
df100.pop("Precip")

raw5 = df5.copy()
raw10 = df10.copy()
raw20 = df20.copy()
raw50 = df50.copy()
raw100 = df100.copy()

raw5.pop("InSitu")
raw10.pop("InSitu")
raw20.pop("InSitu")
raw50.pop("InSitu")
raw100.pop("InSitu")

2        0.20708
3        0.20679
4        0.20750
5        0.20929
6        0.20937
          ...   
42497    0.39700
42498    0.39700
42499    0.39700
42500    0.39700
42501    0.39711
Name: InSitu, Length: 42500, dtype: float64

In [ ]:
def optimizeModel(X, y):
	param_test = {'subsample': [0.8, 0.9, 1],
								'colsample_bytree': [0.8, 0.9, 1],
								'reg_alpha': [0, 1e-1, 5e-1, 1, 3, 5],
								'reg_lambda': [0, 1e-1, 5e-1, 1, 3, 5],
								'learning_rate' :[0.01, 0.05, 0.10, 0.20],
                'n_estimators': [100, 200, 300, 400],
								'max_depth' : [3,5,7,10]}
	
	rgr = xgb.XGBRegressor(max_depth=-1, random_state=42, silent=True, metric='mse', n_jobs=-1, n_estimators = 100)
	
	gs = RandomizedSearchCV(estimator=rgr, param_distributions=param_test, n_iter=15, random_state=314, verbose=True, refit = True)
	gs.fit(X, y)
	print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))
	opt_params = gs.best_params_
        
	return opt_params

In [ ]:
# evaluate xgb ensemble for regression for 5cm
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
# define dataset
train_size_5 = int(len(df5)*0.9)
train_dataset_5, test_dataset_5 = df5.iloc[:train_size_5], df5.iloc[train_size_5:]

X5_train = np.asarray(train_dataset_5.drop('InSitu', axis = 1))
y5_train = np.asarray(train_dataset_5.loc[:,['InSitu']])
y5_train.reshape(-1, 1)

# standard scaling
input_scaler_5 = StandardScaler().fit(X5_train)
X5_train = input_scaler_5.transform(X5_train)
# output_scaler_5 = StandardScaler().fit(y5_train)
# y5_train = output_scaler_5.transform(y5_train)
# define the model
opt_params = optimizeModel(X5_train, y5_train.ravel())
model = xgb.XGBRegressor()
model.set_params(**opt_params)
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X5_train, y5_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Depth 5 MSE: %.4f (%.4f)' % (np.mean(n_scores), np.std(n_scores)))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  7.3min finished


Best score reached: 0.36335693130640395 with params: {'subsample': 0.8, 'reg_lambda': 3, 'reg_alpha': 1, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.9} 
Depth 5 MSE: -0.0013 (0.0000)


In [ ]:
# evaluate xgb ensemble for regression for 10cm
# define dataset
train_size_10 = int(len(df10)*0.9)
train_dataset_10, test_dataset_10 = df10.iloc[:train_size_10], df10.iloc[train_size_10:]

X10_train = np.asarray(train_dataset_10.drop('InSitu', axis = 1))
y10_train = np.asarray(train_dataset_10.loc[:,['InSitu']])
y10_train.reshape(-1, 1)

# standard scaling
input_scaler_10 = StandardScaler().fit(X10_train)
X10_train = input_scaler_10.transform(X10_train)
# output_scaler_10 = StandardScaler().fit(y10_train)
# y10_train = output_scaler_10.transform(y10_train)
# define the model
opt_params = optimizeModel(X10_train, y10_train.ravel())
model = xgb.XGBRegressor()
model.set_params(**opt_params)
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X10_train, y10_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Depth 10 MSE: %.4f (%.4f)' % (np.mean(n_scores), np.std(n_scores)))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  7.0min finished


Best score reached: 0.33958845224497003 with params: {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 0, 'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.05, 'colsample_bytree': 0.8} 
Depth 10 MSE: -0.0011 (0.0000)


In [ ]:
# evaluate xgb ensemble for regression for 20cm
# define dataset
train_size_20 = int(len(df20)*0.9)
train_dataset_20, test_dataset_20 = df20.iloc[:train_size_20], df20.iloc[train_size_20:]

X20_train = np.asarray(train_dataset_20.drop('InSitu', axis = 1))
y20_train = np.asarray(train_dataset_20.loc[:,['InSitu']])
y20_train.reshape(-1, 1)

# standard scaling
input_scaler_20 = StandardScaler().fit(X20_train)
X20_train = input_scaler_20.transform(X20_train)
# output_scaler_20 = StandardScaler().fit(y20_train)
# y20_train = output_scaler_20.transform(y20_train)
# define the model
opt_params = optimizeModel(X20_train, y20_train.ravel())
model = xgb.XGBRegressor()
model.set_params(**opt_params)
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X20_train, y20_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Depth 20 MSE: %.4f (%.4f)' % (np.mean(n_scores), np.std(n_scores)))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  6.4min finished


Best score reached: 0.28595467177610684 with params: {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 0.5, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8} 
Depth 20 MSE: -0.0015 (0.0000)


In [ ]:
# evaluate xgb ensemble for regression for 50cm
# define dataset
train_size_50 = int(len(df50)*0.9)
train_dataset_50, test_dataset_50 = df50.iloc[:train_size_50], df50.iloc[train_size_50:]

X50_train = np.asarray(train_dataset_50.drop('InSitu', axis = 1))
y50_train = np.asarray(train_dataset_50.loc[:,['InSitu']])
y50_train.reshape(-1, 1)

# standard scaling
input_scaler_50 = StandardScaler().fit(X50_train)
X50_train = input_scaler_50.transform(X50_train)
# output_scaler_50 = StandardScaler().fit(y50_train)
# y50_train = output_scaler_50.transform(y50_train)
# define the model
opt_params = optimizeModel(X50_train, y50_train.ravel())
model = xgb.XGBRegressor()
model.set_params(**opt_params)
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X50_train, y50_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Depth 50 MSE: %.4f (%.4f)' % (np.mean(n_scores), np.std(n_scores)))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  5.6min finished


Best score reached: 0.14103964777002748 with params: {'subsample': 1, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.05, 'colsample_bytree': 0.9} 
Depth 50 MSE: -0.0002 (0.0000)


In [ ]:
# evaluate xgb ensemble for regression for 100cm
# define dataset
train_size_100 = int(len(df100)*0.9)
train_dataset_100, test_dataset_100 = df100.iloc[:train_size_100], df100.iloc[train_size_100:]

X100_train = np.asarray(train_dataset_100.drop('InSitu', axis = 1))
y100_train = np.asarray(train_dataset_100.loc[:,['InSitu']])
y100_train.reshape(-1, 1)

# standard scaling
input_scaler_100 = StandardScaler().fit(X100_train)
X100_train = input_scaler_100.transform(X100_train)
# output_scaler_100 = StandardScaler().fit(y100_train)
# y100_train = output_scaler_100.transform(y100_train)
# define the model
opt_params = optimizeModel(X100_train, y100_train.ravel())
model = xgb.XGBRegressor()
model.set_params(**opt_params)
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X100_train, y100_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Depth 100 MSE: %.4f (%.4f)' % (np.mean(n_scores), np.std(n_scores)))

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  5.2min finished


Best score reached: -0.09889935590928178 with params: {'subsample': 0.8, 'reg_lambda': 0.5, 'reg_alpha': 0.5, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8} 
Depth 100 MSE: -0.0018 (0.0001)
